In [1]:
# 线性回归的目标:z=(<x,w> - y)^2 最小
%matplotlib inline
import random
import torch
import pandas as pd
import math
from pymodel.qmodel import *

In [2]:
# 示例:温度为25°C,水的浓度为10g/m^3,气压为101.325kPa(对应标准大气压)
T = 25  # °C
H2O_concentration = 20  # g/m^3
P = 101.325  # kPa

RH = relative_humidity(T, H2O_concentration, P)
print("相对湿度为:", RH, "%")

VPD = vapor_pressure_deficit(T, H2O_concentration, P)
print("蒸汽压差为:", VPD, "kPa")

相对湿度为: 62.72482926324863 %
蒸汽压差为: 1.1806647303049318 kPa


In [43]:
observation_df = pd.read_csv('./data/A_ci.csv')
observation_df = observation_df[observation_df['Photo'] > 0]

observation_df['RH'] = relative_humidity(observation_df['Tair'],observation_df['H2OS'],observation_df['Press'])
observation_df = observation_df[observation_df['RH'] <= 100]
observation_df = observation_df[observation_df['CO2S'] <= 450]
observation_df = observation_df[observation_df['CO2S'] >= 250]

Photo = observation_df['Photo'].tolist()
Tair = observation_df['Tair'].tolist()
CO2S = observation_df['CO2S'].tolist()
RH = (observation_df['RH']/100).tolist()
PARi = observation_df['PARi'].tolist()
Press = observation_df['Press'].tolist()

labels = torch.tensor(Photo)
features = torch.tensor([Tair,CO2S,RH,PARi,Press]).T


In [20]:
# 定义模型
leaf = Leaf()
# 定义模型和初始化参数
leaf = Leaf()
initial_params = torch.tensor([0.11, 0.8, 0.001])


def fit_qmodel(X, w):  #@save
    """线性回归模型"""
    result_list = []
    
    for i in range(0,len(X)):
        An_dict = Q_model(leaf = leaf,
                        Tair= X[i][0],co2 = X[i][1],RH = X[i][2],PAR = X[i][3],patm = X[i][4],
                        cost = w)
        result_list.append(An_dict['An'][0])

    return torch.tensor(result_list)

def run_qmodel(X, w):  #@save
    """线性回归模型"""
    results = pd.DataFrame()
    
    for i in range(0,len(X)):
        An_dict = Q_model(leaf = leaf,
                        Tair= X[i][0],co2 = X[i][1],RH = X[i][2],PAR = X[i][3],patm = X[i][4],
                        cost = w)
        results = pd.concat([results,pd.DataFrame(An_dict)], ignore_index=True)    
    return results
# 定义模型和初始化参数
leaf = Leaf()
initial_params = torch.tensor([0.11, 0.8, 0.001])

# 定义损失函数
def loss_fn(y_hat, y):
    return torch.mean((y_hat - y) ** 2)

In [21]:
# 定义蒙特卡罗采样次数
num_samples = 10

lr = 0.03
batch_size = 100

# 初始化参数估计列表
parameter_estimates = []

initial_params = torch.tensor([0.11, 0.8, 0.001])
best_loss = 1000
best_params = torch.tensor([0.11, 0.8, 0.001])
i = 0
# 进行蒙特卡罗采样和参数估计
while i < 100:
    # 从参数空间中随机采样一组参数值
    sampled_params = initial_params +  torch.randn_like(initial_params) * lr
    if sum(sampled_params>0)==3:        
        i+=1
        # 计算模型的预测输出
        y_hat = fit_qmodel(features, sampled_params)
        
        # 计算损失函数值
        loss = loss_fn(y_hat, labels)
        if best_loss > loss.item():
            best_loss = loss.item()
            best_params = sampled_params
        # 将参数值和损失函数值添加到列表中
        parameter_estimates.append((sampled_params, loss.item()))    

# 计算参数的期望值
[best_params,best_loss]


KeyboardInterrupt: 

In [22]:
[best_params,best_loss]

[tensor([0.1332, 0.7541, 0.0250]), 14.709868431091309]

In [57]:
observation_df[['Photo','Tair','CO2S','RH','PARi','Press']]

,Photo,Tair,CO2S,RH,PARi,Press
114,5.775690,21.998484,382.220306,96.759742,1299.161987,100.509232
118,5.487268,30.219215,383.191406,88.464647,512.032349,99.821251
121,4.277795,30.203333,386.087280,88.235996,510.285736,99.825935
123,3.306961,30.262617,295.724091,88.202335,501.878601,99.711052
124,4.506136,30.244951,384.470398,88.182835,501.507111,99.716270
...,...,...,...,...,...,...
33527,1.612717,30.169411,386.972046,39.872412,1299.636719,101.145882
33533,1.557725,30.184185,387.547668,39.582874,1299.809692,101.144753
33615,5.158652,30.683886,382.153198,8.920464,1299.370117,100.938538
33616,3.743112,30.652998,293.566315,8.825246,1300.287598,100.939949


In [62]:
initial_params = torch.tensor([0.11,0.8,0.00001])
result = run_qmodel(features, initial_params)
result

,An,cp,Ci,Vc,E,gs
0,tensor(6.5345),42.75,tensor(268.4945),tensor(73.1221),tensor(0.0463),tensor(733.2093)
1,tensor(6.6655),42.75,tensor(278.7556),tensor(55.0171),tensor(0.0901),tensor(496.1436)
2,tensor(6.9102),42.75,tensor(279.4036),tensor(56.7239),tensor(0.0933),tensor(509.8915)
3,0,42.75,tensor(294.8696),0,0,0
4,tensor(6.5531),42.75,tensor(280.3612),tensor(53.7228),tensor(0.0900),tensor(482.4094)
...,...,...,...,...,...,...
6733,tensor(5.9901),42.75,tensor(230.1251),tensor(70.8254),tensor(0.0416),tensor(237.2173)
6734,tensor(5.9378),42.75,tensor(230.1402),tensor(70.3563),tensor(0.0414),tensor(233.2257)
6735,0,42.75,tensor(385.7399),0,0,0
6736,0,42.75,tensor(296.3257),0,0,0
